# Prediction of Regulator Function 

*Author: Julian Borbeck*

This notebook contains the documentation of analyses done on LysR family transcription factors as a model for prokaryotic Transcription.

Regulators of the LysR family are among the most abundant prokaryotic transcription factors<cite>[1]</cite>. They can also act as either global or local regulators which can repress or activate the transcription of genes<cite>[1]</cite>. The goal of this study was to investigate the relation of function and phylogeny of these regulators, aswell as create a model to predict the function of LysR family regulators based on sequence information.

## Obtaining Data

Sequences of LysR family regulators with added information about the function of these regulators were obtained from www.uniprot.org/ using the following searches:

* https://www.uniprot.org/uniprot/?query=keyword%3A%22Activator+%5BKW-0010%5D%22+LysR&sort=score
* https://www.uniprot.org/uniprot/?query=keyword%3A%22Repressor+%5BKW-0678%5D%22+LysR&sort=score

Data was obtained as two .gff files. Information about the position of the LysR domain was retrieved from these files, alongside the Uniprot Protein Identifiers.
The LysR domain is the domain, which interacts closely with the DNA. With the Protein identifiers and Domain positions the FASTA sequences of the LysR domains were retrieved using the Uniprot Retrieve/ID mapping service.

[1] Maddocks, S.; Oyston, P.; Microbiology 2008. 154:3609–3623

## Prepare data for Uniprot Retrieve/ID mapping

In [1]:
import pandas as pd
import math
import os

# import .gff files obtained from uniprot into pandas dataframe
path = os.path.dirname(__file__)
Activator = pd.read_csv(os.path.join(path,"uniprotActivator_all.gff"), sep= "\t",header= None)
Repressor = pd.read_csv(os.path.join(path,"uniprotRepressor_all.gff"), sep="\t",header= None)



# instance list of Activators and Repressors for export
Activators =[]
Repressors =[]

# retrieve relevant data from Dataframe
for i in range(0,len(Activator)):

    name = str(Activator[0][i]) #get regulator ID
    
    #add Domain sequence information
    name +="["
    if(not math.isnan(Activator[3][i])): #check for empty dataframe cell
        name += str(int(Activator[3][i]))
    name += "-"
    if (not math.isnan(Activator[4][i])):
        name += str(int(Activator[4][i]))
    name += "]"
    function = Activator[8][i]
    
    # check if domain is part of the LysR family
    if( "HTH lysR-type" in str(function)):
        if(name not in Activators):
            Activators +=[name]

for i in range(0,len(Repressor)):

    name = str(Repressor[0][i])
    name += "["
    if (not math.isnan(Repressor[3][i])):
        name += str(int(Repressor[3][i]))
    name += "-"
    if (not math.isnan(Repressor[3][i])):
        name += str(int(Repressor[4][i]))
    name += "]"
    function = Repressor[8][i]

    if ("HTH lysR-type" in str(function)):
        if(name not in Repressors):
            Repressors +=[name]
            
# print length of list of Activators and Repressors
print(len(Activators),len(Repressors))

#check for duplicates in list to add to list of bifunctional Proteins

duplicates =[]

# check if ID in both Activator and Repressor list
for i in range(0,len(Activators)):
    name = Activators[i]
    for j in range(0,len(Repressors)):
        name1 = Repressors[j]
        if(name == name1):
            duplicates += [name]
            
# print length of duplicate list
print(len(duplicates))

# remove duplicates from both Activator and Repressor lists
for i in range(0,len(duplicates)):
    name = duplicates[i]
    for j in range(0,len(Activators)):
        name1 = Activators[j]
        if(name == name1):
            del Activators[j]
            break
    for j in range(0,len(Repressors)):
        name1 = Repressors[j]
        if(name == name1):
            del Repressors[j]
            break
            
# print length of Activator and Repressor list after removal
print(len(Activators),len(Repressors))

# function to convert list of IDs into string for export
def makestring(list):
    string = ""
    for i in range(0,len(list)):
        string += str(list[i])
        string +="\n"
    return string

# make string from list for export
StrRepressor = makestring(Repressors)
StrActivator = makestring(Activators)
StrBoth = makestring(duplicates)

# export
with open(os.path.join(path,"Activators_uniprot.txt"),"w") as f:
    f.write(StrActivator)
with open(os.path.join(path,"Repressors_uniprot.txt"),"w") as f:
    f.write(StrRepressor)
with open(os.path.join(path,"Dual_uniprot.txt"),"w") as f:
    f.write(StrBoth)

12760 1340
59
12701 1281


# Phylogenetic analysis

Due to the ammount of computational power needed for analysis of all Regulators only 4000 were used in the construction of a phylogenetic tree of the LysR domains. 4000 regulators were randomly selected. All FASTA sequences were manually combined in one file.

## Randomly select 4000 sequences for a multiple sequence alignment

In [2]:
import pandas as pd
import random

#initialize seed 
random.seed(111)

# import all 
path = os.path.dirname(__file__)
with open(os.path.join(path,"uniprot_full_all.fasta")) as f:
    fasta = f.read()

# create list of regulators
regulators = fasta.split(">")

# number of regulators for analysis
no_regs = 4000

# create string for export
fasta1 =""

#randomly select 4000 unique regulators by removing from source list
for i in range(0,4000):
    
    random.shuffle(regulators)
    seq = regulators.pop(random.randint(0,len(regulators)))
    fasta1 +=">"
    fasta1 += seq

#export as fasta file
with open(os.path.join(path,"LysR_sample.fasta"),"w") as f1:
    f1.write(fasta1)

## Multiple sequence alignment

Sequences were aligned using ClustalW version 1.2.4 at the EMBL-EBI webserver using the following parameters:
* Output guide tree
    * true 
* Output distance matrix
    * false 
* Dealign input sequences
    * false 
* mBed-like clustering guide tree
    * true 
* mBed-like clustering iteration
    * true 
* Number of iterations
    * 0 
* Maximum guide tree iterations
    * -1 
* Maximum HMM iterations
    * -1 
* Output alignment format
    * fa 
* Output order
    * aligned 
* Sequence Type
    * protein 
The resulting multiple sequence aligment was saved as *MSA_LysR.fasta*

## Tree construction

Using the aligment created in the last step a maximum likelyhood phylogenetic tree was created using MegaX.
This tree was created using the following parameters:
* Test of phylogeny
    * None
* Substitution Model
    * Jones Taylor Thornton model
* ML Heuristic Method
    * Nearest Neighbor interchange

## Tree visualization
Marked in red are Repressors, marked in green are Activators. Blue proteins are both Repressor and Activator.

In [3]:
from ete3 import Tree, faces, AttrFace, TreeStyle, NodeStyle # ete3 is used for tree visualization
import pandas as pd
import math
import matplotlib.pyplot as plt

path = os.path.dirname(__file__)
myfile = "MSA_LysR.fasta"

lookup = pd.read_csv(os.path.join(path,myfile),sep='\t')

#import Tree
t = Tree( os.path.join(path,"LysR_ML.nwk"))

# Uses Activators and Repressors list created as in step 1 as lookup tables

Activator = pd.read_csv(os.path.join(path,"uniprotActivator_all.gff"), sep= "\t",header= None)
Repressor = pd.read_csv(os.path.join(path,"uniprotRepressor_all.gff"), sep="\t",header= None)

Activators =[]
Repressors =[]


for i in range(0,len(Activator)):

    name = str(Activator[0][i])


    function = Activator[8][i]

    if( "HTH lysR-type" in str(function)):
        if(name not in Activators):
            Activators +=[name]

for i in range(0,len(Repressor)):

    name = str(Repressor[0][i])

    function = Repressor[8][i]

    if ("HTH lysR-type" in str(function)):
        if(name not in Repressors):
            Repressors +=[name]

duplicates =[]
for i in range(0,len(Activators)):
    name = Activators[i]
    for j in range(0,len(Repressors)):
        name1 = Repressors[j]
        if(name == name1):
            duplicates += [name]


for i in range(0,len(duplicates)):
    name = duplicates[i]
    for j in range(0,len(Activators)):
        name1 = Activators[j]
        if(name == name1):
            del Activators[j]
            break
    for j in range(0,len(Repressors)):
        name1 = Repressors[j]
        if(name == name1):
            del Repressors[j]
            break



# set styles for nodes
Activator_Style = NodeStyle()
Activator_Style["bgcolor"] = "Green"

Repressor_Style = NodeStyle()
Repressor_Style["bgcolor"] = "Red"

Dual_Style = NodeStyle()
Dual_Style["bgcolor"] = "Blue"

print("constructing tree")
# set nodestyle depending on Function
for n in t.traverse():

    if n.is_leaf():
        
        name = str(n.name).split("|")[1] # set name
        
        if(name in Activators):
            n.set_style(Activator_Style)
        if (name in Repressors):
            n.set_style(Repressor_Style)
        if (name in duplicates):
            n.set_style(Dual_Style)
        n.name = name
        
circular_style = TreeStyle()
circular_style.mode = "c" # draw tree in circular mode

#render Tree
print("done")
t.render(os.path.join(path,"LysR.pdf"), tree_style =circular_style)
t.show(tree_style =circular_style)

constructing tree
done


# Classifier

To classify a sequence as either a Repressor or Activator, Hidden Markov Models were trained. One model was trained on the Repressor sequences, the other on the Activator sequences. Implementation and training of the Hidden Markov Models were done using the python **hmmlearn** library. To predict the function of a gene the models were scored using the score() function. The resulting log likelihood was normalized to the size of the training sample and converted into the likelihood. These likelihoods were compared and a classification was assigned based on comparing the normalized likelihoods.

75 percent of the dataset were used for training, 25 percent were used for model validation.

Both models were trained for 3000 iterations assuming 14 underlying components.

The data was encoded using a numerical system in which the FASTA amino acid code was substituted using integer numbers from 0 to 20.

Resulting models were exported using the python **pickle** library.

In [5]:
import numpy as np
from hmmlearn import hmm
import random
import pickle
from datetime import datetime
import json
import requests
import math


# encode FASTA
def encode(string):
    As = -1
    asseq = []
    for i in range(0,len(string)):
        if(string[i]=="A"):
            As = 0
        elif(string[i]=="C"):
            As = 1
        elif (string[i] == "D"):
            As = 2
        elif (string[i] == "E"):
            As = 3
        elif (string[i] == "F"):
            As = 4
        elif (string[i] == "G"):
            As = 5
        elif (string[i] == "H"):
            As = 6
        elif (string[i] == "I"):
            As = 7
        elif (string[i] == "K"):
            As = 8
        elif (string[i] == "L"):
            As = 9
        elif (string[i] == "M"):
            As = 10
        elif (string[i] == "N"):
            As = 11
        elif (string[i] == "P"):
            As = 12
        elif (string[i] == "Q"):
            As = 13
        elif (string[i] == "R"):
            As = 14
        elif (string[i] == "S"):
            As = 15
        elif (string[i] == "T"):
            As = 16
        elif (string[i] == "V"):
            As = 17
        elif (string[i] == "W"):
            As = 18
        elif (string[i] == "Y"):
            As = 19
        elif (string[i] == "-"):
            As = 20
        asseq +=[As]

    return(asseq)

iterations = 3000
n_components = 14

np.random.seed(111)
random.seed(111)

# import data
path = os.path.dirname(__file__)

Activator="uniprotActivator_all.fasta"
Repressor = "uniprotRepressor_all.fasta"
now = datetime.now() # get timestamp

print("started: "+ str(now))

# get sequences
with open(os.path.join(path,Activator)) as f:
    Activators = f.read()
Activators = Activators.split(">")

with open(os.path.join(path,Repressor)) as f1:
    Repressor = f1.read()
Repressors = Repressor.split(">")

Actseq = []
Actlen = []
Repseq = []
Replen = []

#encode data
for i in range(1,len(Activators)):
    t1 = Activators[i].split("\n")
    s =""
    for j in range(1,len(t1)):
        s += str(t1[j])
    s1 = encode(s)
    Actseq +=[s1]
    Actlen +=[len(s1)]



for i in range(1,len(Repressors)):
    t1 = Repressors[i].split("\n")
    s =""
    for j in range(1,len(t1)):
        s += str(t1[j])

    s1 = encode(s)
    Repseq +=[s1]
    Replen += [len(s1)]



#split sets into train and test sets
trlena = int(len(Actseq)*0.75)
trlenr = int(len(Repseq)*0.75)

testas = []
testal = []

original = len(Actseq)-trlena

# remove random sequences for testing
for i in range(0,original):
    index = random.randint(0,len(Actseq)-1)
    testas += [Actseq.pop(index)]
    testal += [Actlen.pop(index)]

testrs = []
testrl = []
original = len(Repseq)-trlenr
for i in range(0,original):
    index = random.randint(0,len(Repseq)-1)
    testrs += [Repseq.pop(index)]
    testrl += [Replen.pop(index)]



print("starting Training of Activator HMM")

#train models
HMMact = hmm.GaussianHMM(n_components=n_components, covariance_type="full", n_iter=iterations)
act = []
for i in range(0,len(Actseq)):
    se = Actseq[i]
    act += se

npActseq = np.array(act)
npActseq = np.vstack(npActseq)
HMMact.fit(npActseq,Actlen)


print("starting Training of Repressor HMM")


HMMrep = hmm.GaussianHMM(n_components=n_components, covariance_type="full", n_iter=iterations)
rep = []
for i in range(0,len(Repseq)):
    se = Repseq[i]
    rep += se

npRepseq = np.array(rep)
npRepseq = np.vstack(npRepseq)

HMMrep.fit(npRepseq,Replen)
print("Done")

time  = str(datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p"))

with open(os.path.join(path,str(time)+"_Activator.pkl"), "wb") as file:
    pickle.dump(HMMact, file)
with open(os.path.join(path,str(time)+"_Repressor.pkl"), "wb") as file1:
    pickle.dump(HMMrep, file1)


started: 2021-03-21 21:48:42.567194
starting Training of Activator HMM
starting Training of Repressor HMM
Done


In [24]:
# encode FASTA
def encode(string):
    As = -1
    asseq = []
    for i in range(0,len(string)):
        if(string[i]=="A"):
            As = 0
        elif(string[i]=="C"):
            As = 1
        elif (string[i] == "D"):
            As = 2
        elif (string[i] == "E"):
            As = 3
        elif (string[i] == "F"):
            As = 4
        elif (string[i] == "G"):
            As = 5
        elif (string[i] == "H"):
            As = 6
        elif (string[i] == "I"):
            As = 7
        elif (string[i] == "K"):
            As = 8
        elif (string[i] == "L"):
            As = 9
        elif (string[i] == "M"):
            As = 10
        elif (string[i] == "N"):
            As = 11
        elif (string[i] == "P"):
            As = 12
        elif (string[i] == "Q"):
            As = 13
        elif (string[i] == "R"):
            As = 14
        elif (string[i] == "S"):
            As = 15
        elif (string[i] == "T"):
            As = 16
        elif (string[i] == "V"):
            As = 17
        elif (string[i] == "W"):
            As = 18
        elif (string[i] == "Y"):
            As = 19
        elif (string[i] == "-"):
            As = 20
        asseq +=[As]

    return(asseq)

np.random.seed(111)
random.seed(111)

# import data
path = os.path.dirname(__file__)

Activator="uniprotActivator_all.fasta"
Repressor = "uniprotRepressor_all.fasta"
now = datetime.now() # get timestamp

print("started: "+ str(now))

# get sequences
with open(os.path.join(path,Activator)) as f:
    Activators = f.read()
Activators = Activators.split(">")

with open(os.path.join(path,Repressor)) as f1:
    Repressor = f1.read()
Repressors = Repressor.split(">")

Actseq = []
Actlen = []
Repseq = []
Replen = []

#encode data
for i in range(1,len(Activators)):
    t1 = Activators[i].split("\n")
    s =""
    for j in range(1,len(t1)):
        s += str(t1[j])
    s1 = encode(s)
    Actseq +=[s1]
    Actlen +=[len(s1)]



for i in range(1,len(Repressors)):
    t1 = Repressors[i].split("\n")
    s =""
    for j in range(1,len(t1)):
        s += str(t1[j])

    s1 = encode(s)
    Repseq +=[s1]
    Replen += [len(s1)]



#split sets into train and test sets
trlena = int(len(Actseq)*0.75)
trlenr = int(len(Repseq)*0.75)

testas = []
testal = []

original = len(Actseq)-trlena

# remove random sequences for testing
for i in range(0,original):
    index = random.randint(0,len(Actseq)-1)
    testas += [Actseq.pop(index)]
    testal += [Actlen.pop(index)]

testrs = []
testrl = []
original = len(Repseq)-trlenr
for i in range(0,original):
    index = random.randint(0,len(Repseq)-1)
    testrs += [Repseq.pop(index)]
    testrl += [Replen.pop(index)]



with open(os.path.join(path,str(time)+"_Activator.pkl"), "rb") as file: 
    pickle.load(file)

with open(os.path.join(path,str(time)+"_Repressor.pkl"), "rb") as file: 
    pickle.load(file)
    

# test HMM on real data


same = 0
different = 0
guess = 0
actpred = 0
reppred = 0
rightact = 0
wrongact = 0
rightrep = 0
wrongrep = 0
testdata = testas + testrs
for i in range(0,len(testdata)):

    if(len(testas)>0):
        data = 0
    else:
        data = 1
    
    
    if(data == 0):
        index = random.randrange(0,len(testas))
        seq = testas.pop(index)
        seq = np.array(seq)
        seq = np.vstack(seq)
        
    else:
        index = random.randrange(0,len(testrs))
        seq = testrs.pop(index)
        seq = np.array(seq)
        seq = np.vstack(seq)
        
    #average likelyhood
    scoreAct = math.exp(HMMact.score(seq)/trlena)
    scoreRep = math.exp(HMMrep.score(seq)/trlenr)
    
    
    if(scoreRep> scoreAct):
        prediction = 1
        reppred += 1
    else:
        prediction = 0
        actpred += 1
    
    if(data == prediction):
        same +=1
        if(data == 0):
            rightact += 1
        if (data == 1):
            rightrep += 1
    else:
        different += 1
        if (data == 0):
            wrongact += 1
        if (data == 1):
            wrongrep += 1
    


now = datetime.now()

sensitivityrep = rightrep/(rightrep+wrongrep)
specificityrep = rightact/(rightact+wrongact)

pospredvalrep = rightrep/(rightrep+wrongact)
negpredvalrep = rightact/(rightact+wrongrep)

print("finished: " +str(now))
print("--------------------------------------------")
print("Predicted right: " +str(same)+ "\nPredicted wrong: "+ str(different)+"\nPredicted Activator right: "+ str(rightact)+"\nPredicted Repressor right: "+ str(rightrep))
print("Predicted Activator wrong: "+str(wrongact)+"\nPredicted Repressor wrong: "+str(wrongrep))
print("Repression sensitivity: "+str(sensitivityrep)+"\nRepression specificity: "+str(specificityrep))
print("Positive predictive value Repression: "+str(pospredvalrep)+"\nNegative predictive value Repression: "+ str(negpredvalrep))

started: 2021-03-22 00:13:56.479822
finished: 2021-03-22 00:14:04.275652
--------------------------------------------
Predicted right: 3477
Predicted wrong: 20
Predicted Activator right: 3176
Predicted Repressor right: 301
Predicted Activator wrong: 0
Predicted Repressor wrong: 20
Repression sensitivity: 0.9376947040498442
Repression specificity: 1.0
Positive predictive value Repression: 1.0
Negative predictive value Repression: 0.9937421777221527
